# Importando as bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pymannkendall import original_test
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.tsa.seasonal import seasonal_decompose
from db import connection_db as conndb
from db import filters
from utils.extracao_interacao import ExtracaoInteracao
from sklearn.metrics import davies_bouldin_score

# Conectando ao banco de dados do MongoDB

Função para conectar ao MongoDB

In [ ]:
def conectar_mongodb(uri, db_name, collection_name):
    mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=collection_name)
    mongo_connection.connect()
    return mongo_connection.collection

In [ ]:
# Definindo os parâmetros de conexão
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
collection_name = 'dadosSemFiltros'
collection_name_filtered = 'posts7anos1anos'

collection = conectar_mongodb(uri, db_name, collection_name)
collection_filtrada = conectar_mongodb(uri, db_name, collection_name_filtered)

# Aplicando pipeline para preparação dos dados

Função para aplicar os pipelines de filtro

In [ ]:
def aplicar_pipelines(collection, collection_filtrada):
    collection_filters = filters.CollectionFilters(collection)
    collection_filters_likes = filters.CollectionFilters(collection_filtrada)

    # Aplicar pipelines
    collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
    collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')
    collection_filters.apply_pipeline3('postsComBDIAndInfos')
    collection_filters.apply_pipeline4('posts')
    collection_filters_likes.apply_pipeline6('likes')

    return collection_filters.collection, collection_filters_likes.collection

In [ ]:
# Aplicando os pipelinesc
collection, collection_filtrada = aplicar_pipelines(collection, collection_filtrada)

# Carregando os dados em um dataframe

In [ ]:
documentos = collection.find()
df_post_original = pd.DataFrame(list(documentos))

# Visualizar as primeiras linhas do dataframe
df_post_original.head()

In [ ]:
documentos_likes = collection_filtrada.find()
df_original_likes = pd.DataFrame(list(documentos_likes))

# Visualizar as primeiras linhas do dataframe
df_original_likes.head()

In [ ]:
# Transformando a coluna sexo em binária
df_post_original['sexo'] = df_post_original['sexo'].map({'F': 0, 'M': 1})

# Removendo colunas desnecessárias
df_post_original.drop(columns=['_id', 'diaDaSemana'], inplace=True)

# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima', 'critica', 'suicida', 'choro',
    'agitacao', 'interesse', 'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]


# Função para preencher valores nulos e converter o tipo de dado
def preencher_e_converter(df, colunas, valor_preenchimento=0, tipo_dados='int64'):
    df[colunas] = df[colunas].fillna(valor_preenchimento)
    df[colunas] = df[colunas].astype(tipo_dados)
    return df

In [ ]:
df_post_original = preencher_e_converter(df_post_original, colunas_para_converter)

In [ ]:
# Remover usuarios que tem menos de 100 posts
df_post_original = df_post_original.groupby('id_usuario').filter(lambda value: len(value) >= 100)

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_posts(df):
    df['data'] = df['postCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]
    df_data = df_data[df_data['suicida'] == 3]

    # Remover posts vazios
    df_data = df_data.dropna(subset=['postMessage', 'postStory'], how='all')
    df_data = df_data[~((df_data['postMessage'].str.strip() == '') & (df_data['postStory'].str.strip() == ''))]

    return df_data

In [ ]:
df_posts = manipular_e_filtrar_posts(df_post_original)

# Remover usuarios que tem menos de 100 posts
df_posts = df_posts.groupby('id_usuario').filter(lambda value: len(value) >= 100)

df_posts.head()

In [ ]:
# Função para manipular datas e filtrar posts
def manipular_e_filtrar_likes(df):
    df['data'] = df['likeCreatedTime'].dt.date
    df['data'] = pd.to_datetime(df['data'])
    df['mes'] = df['data'].dt.to_period('M')
    df['semana'] = df['data'].dt.to_period('W')

    df_data = df[df['data'].dt.year == 2017]
    df_data = df_data[~((df_data['data'].dt.month == 12) & (df_data['data'].dt.year == 2017))]
    df_data = df_data[df_data['data'].dt.month >= 5]

    return df_data

In [ ]:
df_likes = manipular_e_filtrar_likes(df_original_likes)

# Pegar os mesmos usuários do df_posts
df_likes = df_likes[df_likes['id_usuario'].isin(df_posts['id_usuario'])]

# Adicionar a coluna de quantidade de likes
df_likes['quantLikes'] = df_likes['id_usuario'].apply(lambda value: 1 if pd.notnull(value) else 0)

df_likes.head()

# Adicionar interações do usuário

In [ ]:
df_posts['quantPostMsg'] = df_posts['postMessage'].apply(
    lambda value: 1 if pd.notnull(value) and value.strip() != '' else 0)

df_posts['quantPostStory'] = df_posts['postStory'].apply(
    lambda value: 1 if pd.notnull(value) and value.strip() != '' else 0)

df_posts['quantPosts'] = 1

In [ ]:
extracao = ExtracaoInteracao(df_posts)
df_posts_interacoes = extracao.extract_interactions()

df_posts_interacoes.head()

Função para calcular e visualizar a distribuição de postagens por usuário

In [ ]:
def plotar_histogramas_posts(df):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), dpi=300)

    # Quantidade geral de postagens
    quant_posts = df.groupby('id_usuario').size().reset_index(name='quant_posts')
    axs[0].hist(quant_posts['quant_posts'], bins=50)
    axs[0].set_xlabel('Quant de Posts')
    axs[0].set_ylabel('Quant de Usuários')

    # Quantidade de postagens com mensagem
    quant_msg = df.groupby('id_usuario')['quantPostMsg'].sum().reset_index(name='quant_msg')
    axs[1].hist(quant_msg['quant_msg'], bins=50)
    axs[1].set_xlabel('Quant de Posts com Mensagem')
    axs[1].set_ylabel('Quant de Usuários')

    # Quantidade de postagens com story
    quant_story = df.groupby('id_usuario')['quantPostStory'].sum().reset_index(name='quant_story')
    axs[2].hist(quant_story['quant_story'], bins=50)
    axs[2].set_xlabel('Quant de Posts com Story')
    axs[2].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_posts(df_posts_interacoes)

In [ ]:
def plotar_histogramas_interacoes(df):
    fig, axs = plt.subplots(3, 2, figsize=(10, 8), dpi=300)

    # Quantidade de atualização de Perfil
    quant_profile = df.groupby('id_usuario')['quantProfile'].sum().reset_index(name='quant_profile')
    axs[0, 0].hist(quant_profile['quant_profile'], bins=50)
    axs[0, 0].set_xlabel('Quant de Atualizações de Perfil')
    axs[0, 0].set_ylabel('Quant de Usuários')

    # Quantidade de atualização de Capa
    quant_cover = df.groupby('id_usuario')['quantCover'].sum().reset_index(name='quant_cover')
    axs[0, 1].hist(quant_cover['quant_cover'], bins=50)
    axs[0, 1].set_xlabel('Quant de Atualizações de Capa')
    axs[0, 1].set_ylabel('Quant de Usuários')

    # Quantidade de fotos com outras pessoas
    quant_photo_with_others = df.groupby('id_usuario')['quantAddPhotoWithOthers'].sum().reset_index(
        name='quant_photo_with_others')
    axs[1, 0].hist(quant_photo_with_others['quant_photo_with_others'], bins=50)
    axs[1, 0].set_xlabel('Quant de Fotos com Outras Pessoas')
    axs[1, 0].set_ylabel('Quant de Usuários')

    # Quantidade de está com outras pessoas
    quant_is_with_others = df.groupby('id_usuario')['quantIsWithOthers'].sum().reset_index(name='quant_is_with_others')
    axs[1, 1].hist(quant_is_with_others['quant_is_with_others'], bins=50)
    axs[1, 1].set_xlabel('Quant de Está com Outras Pessoas')
    axs[1, 1].set_ylabel('Quant de Usuários')

    # Quantidade de fotos adicionadas
    quant_add_photo = df.groupby('id_usuario')['quantAddPhoto'].sum().reset_index(name='quant_add_photo')
    axs[2, 0].hist(quant_add_photo['quant_add_photo'], bins=50)
    axs[2, 0].set_xlabel('Quant de Fotos Adicionadas')
    axs[2, 0].set_ylabel('Quant de Usuários')

    # Quantidade de fotos compartilhadas
    quant_shared_photo = df.groupby('id_usuario')['quantSharedPhoto'].sum().reset_index(name='quant_shared_photo')
    axs[2, 1].hist(quant_shared_photo['quant_shared_photo'], bins=50)
    axs[2, 1].set_xlabel('Quant de Fotos Compartilhadas')
    axs[2, 1].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_interacoes(df_posts_interacoes)

In [ ]:
def plotar_histogramas_interacoes_2(df):
    fig, axs = plt.subplots(3, 2, figsize=(10, 8), dpi=300)

    # Quantidade de vídeos compartilhados
    quant_shared_video = df.groupby('id_usuario')['quantSharedVideo'].sum().reset_index(name='quant_shared_video')
    axs[0, 0].hist(quant_shared_video['quant_shared_video'], bins=50)
    axs[0, 0].set_xlabel('Quant de Vídeos Compartilhados')
    axs[0, 0].set_ylabel('Quant de Usuários')

    # Quantidade de links compartilhados
    quant_shared_link = df.groupby('id_usuario')['quantSharedLink'].sum().reset_index(name='quant_shared_link')
    axs[0, 1].hist(quant_shared_link['quant_shared_link'], bins=50)
    axs[0, 1].set_xlabel('Quant de Links Compartilhados')
    axs[0, 1].set_ylabel('Quant de Usuários')

    # Quantidade de posts compartilhados
    quant_shared_post = df.groupby('id_usuario')['quantSharedPost'].sum().reset_index(name='quant_shared_post')
    axs[1, 0].hist(quant_shared_post['quant_shared_post'], bins=50)
    axs[1, 0].set_xlabel('Quant de Posts Compartilhados')
    axs[1, 0].set_ylabel('Quant de Usuários')

    # Quantidade de eventos compartilhados
    quant_shared_event = df.groupby('id_usuario')['quantSharedEvent'].sum().reset_index(name='quant_shared_event')
    axs[1, 1].hist(quant_shared_event['quant_shared_event'], bins=50)
    axs[1, 1].set_xlabel('Quant de Eventos Compartilhados')
    axs[1, 1].set_ylabel('Quant de Usuários')

    # Quantidade de memórias compartilhadas
    quant_shared_memory = df.groupby('id_usuario')['quantSharedMemory'].sum().reset_index(name='quant_shared_memory')
    axs[2, 0].hist(quant_shared_memory['quant_shared_memory'], bins=50)
    axs[2, 0].set_xlabel('Quant de Memórias Compartilhadas')
    axs[2, 0].set_ylabel('Quant de Usuários')

    # Quantidade de status
    quant_status = df.groupby('id_usuario')['quantStatus'].sum().reset_index(name='quant_status')
    axs[2, 1].hist(quant_status['quant_status'], bins=50)
    axs[2, 1].set_xlabel('Quant de Atualizações de Status')
    axs[2, 1].set_ylabel('Quant de Usuários')

    plt.tight_layout()
    plt.show()

In [ ]:
plotar_histogramas_interacoes_2(df_posts_interacoes)

In [ ]:
df_agrupado_soma = df_posts_interacoes.groupby(['id_usuario']).agg(
    quantPosts=('quantPosts', 'sum'),
    somaProfile=('quantProfile', 'sum'),
    somaCover=('quantCover', 'sum'),
    somaAddPhotoWithOthers=('quantAddPhotoWithOthers', 'sum'),
    somaIsWithOthers=('quantIsWithOthers', 'sum'),
    somaAddPhoto=('quantAddPhoto', 'sum'),
    somaSharedPhoto=('quantSharedPhoto', 'sum'),
    somaSharedVideo=('quantSharedVideo', 'sum'),
    somaSharedLink=('quantSharedLink', 'sum'),
    somaSharedPost=('quantSharedPost', 'sum'),
    somaSharedEvent=('quantSharedEvent', 'sum'),
    somaSharedMemory=('quantSharedMemory', 'sum'),
    somaStatus=('quantStatus', 'sum'),
    pessimismo=('pessimismo', 'first'),
    tristeza=('tristeza', 'first'),
    fracasso=('fracasso', 'first'),
    prazer=('prazer', 'first'),
    culpa=('culpa', 'first'),
    punicao=('punicao', 'first'),
    estima=('estima', 'first'),
    critica=('critica', 'first'),
    choro=('choro', 'first'),
    agitacao=('agitacao', 'first'),
    interesse=('interesse', 'first'),
    indecisao=('indecisao', 'first'),
    desvalorizacao=('desvalorizacao', 'first'),
    energia=('energia', 'first'),
    sono=('sono', 'first'),
    irritabilidade=('irritabilidade', 'first'),
    apetite=('apetite', 'first'),
    concentracao=('concentracao', 'first'),
    fadiga=('fadiga', 'first'),
    int_sexo=('int_sexo', 'first'),
    idade=('idade', 'first')
).reset_index()

In [ ]:
df_likes_agrupado = df_likes.groupby('id_usuario').agg(somaLikes=('quantLikes', 'sum'))

# Juntar os dataframes
df_agrupado_soma = df_agrupado_soma.merge(df_likes_agrupado, on='id_usuario', how='left')

# Preencher valores NaN com 0
df_agrupado_soma.fillna(0, inplace=True)

In [ ]:
# Criar uma matriz de correlação
colunas_interacoes = df_agrupado_soma[[
    'somaProfile', 'somaCover', 'somaAddPhotoWithOthers', 'somaIsWithOthers', 'somaAddPhoto', 'somaSharedPhoto',
    'somaSharedVideo', 'somaSharedLink', 'somaSharedPost', 'somaSharedEvent', 'somaSharedMemory', 'somaStatus',
    'somaLikes'
]]

matriz_correlacao = colunas_interacoes.corr(method='spearman')

plt.figure(figsize=(12, 8))
sns.heatmap(matriz_correlacao, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

In [ ]:
from scipy.stats import spearmanr

# Testar para todas as combinações de variáveis
for i in range(len(colunas_interacoes.columns)):
    for j in range(i + 1, len(colunas_interacoes.columns)):
        x = colunas_interacoes.iloc[:, i]
        y = colunas_interacoes.iloc[:, j]

        spearman_corr, p_value = spearmanr(x, y)
        print(
            f'Correlação de Spearman entre {colunas_interacoes.columns[i]} e {colunas_interacoes.columns[j]}: {spearman_corr}')
        print(f'P-valor de Spearman: {p_value}')

In [ ]:
from sklearn.preprocessing import StandardScaler

variaveis = df_agrupado_soma[[
    'quantPosts'
]]

# Normalizar as variáveis
scaler = StandardScaler()
variaveis_normalizadas = scaler.fit_transform(variaveis)

In [ ]:
k_values = range(2, 11)
silhouette_scores = []
davies_bouldin_scores = []
inertias = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(variaveis_normalizadas)
    inertias.append(kmeans.inertia_)

    # Obter os rótulos dos clusters
    labels = kmeans.labels_

    # Calcular o Coeficiente de Silhueta
    silhouette_avg = silhouette_score(variaveis_normalizadas, labels)
    silhouette_scores.append(silhouette_avg)

    # Calcular o Índice de Davies-Bouldin
    db_index = davies_bouldin_score(variaveis_normalizadas, labels)
    davies_bouldin_scores.append(db_index)

In [ ]:
# Plotar as métricas em um único gráfico
fig, ax1 = plt.subplots(figsize=(10, 6))

# Gráfico da Inércia
color_inertia = 'tab:blue'
ax1.set_xlabel('Número de Clusters (k)')
ax1.set_ylabel('Inércia', color=color_inertia)
ax1.plot(k_values, inertias, marker='o', color=color_inertia, label='Inércia')
ax1.tick_params(axis='y', labelcolor=color_inertia)
ax1.set_xticks(k_values)

ax2 = ax1.twinx()

color_silhouette = 'tab:green'
color_db = 'tab:red'
ax2.set_ylabel('Métricas', color='black')
ax2.plot(k_values, silhouette_scores, marker='s', color=color_silhouette, label='Coeficiente de Silhueta')
ax2.plot(k_values, davies_bouldin_scores, marker='^', color=color_db, label='Índice de Davies-Bouldin')
ax2.tick_params(axis='y', labelcolor='black')

# Combinar as legendas
lines_labels = [ax1.get_legend_handles_labels(), ax2.get_legend_handles_labels()]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
ax2.legend(lines, labels, loc='upper right')

# Título do Gráfico
# plt.title('Avaliação do K-Means com Inércia, Coeficiente de Silhueta e Índice de Davies-Bouldin')

plt.show()

In [ ]:
# Printar o silhouette score junto com o número de clusters
for k, score in zip(k_values, silhouette_scores):
    print(f'Número de Clusters: {k} - Coeficiente de Silhueta: {score}')

print('\n')

# Printar o índice de Davies-Bouldin junto com o número de clusters
for k, score in zip(k_values, davies_bouldin_scores):
    print(f'Número de Clusters: {k} - Índice de Davies-Bouldin: {score}')

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)

kmeans.fit(variaveis_normalizadas)

df_agrupado_soma['cluster'] = kmeans.labels_

print(df_agrupado_soma[['id_usuario', 'cluster']].head())

In [ ]:
df_agrupado_soma.to_csv('df_agrupado_soma.csv', index=False)
df_agrupado_soma['cluster'].value_counts()

In [ ]:
def teste_mk(df_cluster, periodo, tipo_serie='posts'):
    resultados_mk = []

    for usuario in df_cluster['id_usuario'].unique():
        df_usuario = df_cluster[df_cluster['id_usuario'] == usuario]

        eventos_por_dia = df_usuario.groupby('data').size()

        if len(eventos_por_dia) >= 2 * periodo:
            try:
                resultado = seasonal_decompose(eventos_por_dia, model='additive', period=periodo)
                tendencia = resultado.trend.dropna()

                if len(tendencia) > 0:
                    result = original_test(tendencia)

                    if hasattr(result, 'trend'):
                        resultados_mk.append({
                            'id_usuario': usuario,
                            'trend': result.trend,
                            'h': result.h,
                            'p-valor': result.p,
                            'z': result.z,
                            'Tau': result.Tau,
                            's': result.s,
                            'var_s': result.var_s,
                            'slope': result.slope,
                            'intercept': result.intercept
                        })
                    else:
                        print(f"Usuário {usuario} - Resultado do teste de Mann-Kendall sem 'trend'.")
                else:
                    print(f"Usuário {usuario} - Não foi possível decompor dados suficientes para tendência.")
            except ValueError as e:
                print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
        else:
            print(
                f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(eventos_por_dia)}")

    # Criando o DataFrame apenas se houver resultados suficientes
    if resultados_mk:
        df_resultados_mk = pd.DataFrame(resultados_mk)

        if 'trend' in df_resultados_mk.columns:
            contagem_tendencias = df_resultados_mk['trend'].value_counts()
            print(contagem_tendencias)

            total_usuarios = df_resultados_mk.shape[0]
            percentuais = (contagem_tendencias / total_usuarios) * 100
            print(percentuais)

            # Gráfico de barras das tendências
            plt.figure(figsize=(8, 6))
            sns.countplot(data=df_resultados_mk, x='trend', order=['increasing', 'decreasing', 'no trend'])
            plt.title(f'Distribuição de Tendências entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Tipo de Tendência')
            plt.ylabel('Número de Usuários')
            plt.show()

            # Gráfico de distribuição dos slopes
            plt.figure(figsize=(10, 6))
            sns.histplot(df_resultados_mk['slope'], bins=20, kde=True)
            plt.title(f'Distribuição dos Slopes entre Usuários ({tipo_serie.capitalize()})')
            plt.xlabel('Slope')
            plt.ylabel('Frequência')
            plt.show()

        else:
            print("Nenhuma tendência foi detectada nos dados.")

        return df_resultados_mk
    else:
        print("Nenhum dado disponível para análise.")
        return pd.DataFrame()

In [ ]:
# buscar no dataframe df_posts_interacoes os usuários do cluster 0, 1 e 2
df_cluster0_soma = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_soma[df_agrupado_soma['cluster'] == 0]['id_usuario'])]
df_cluster1_soma = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_soma[df_agrupado_soma['cluster'] == 1]['id_usuario'])]
df_cluster2_soma = df_posts_interacoes[
    df_posts_interacoes['id_usuario'].isin(df_agrupado_soma[df_agrupado_soma['cluster'] == 2]['id_usuario'])]

In [ ]:
resultados_mk_cluster0 = teste_mk(df_cluster0_soma, 30)
resultados_mk_cluster1 = teste_mk(df_cluster1_soma, 30)
resultados_mk_cluster2 = teste_mk(df_cluster2_soma, 30)

In [59]:
# Filtrar o DataFrame para manter apenas os usuários com cluster igual a 1
df_cluster_0 = df_agrupado_soma[df_agrupado_soma['cluster'] == 0]
df_cluster_0.describe()

,quantPosts,somaProfile,somaCover,somaAddPhotoWithOthers,somaIsWithOthers,somaAddPhoto,somaSharedPhoto,somaSharedVideo,somaSharedLink,somaSharedPost,...,apetite,concentracao,fadiga,int_sexo,idade,somaLikes,mediaInteracoesComOutros,mediaSharedPhotoPostVideo,mediaAtualizacaoPerfil,cluster
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,...,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0
mean,792.052632,4.052632,0.631579,4.684211,20.473684,10.368421,242.631579,104.789474,13.947368,288.157895,...,2.473684,2.105263,2.473684,1.578947,21.526316,244.105263,12.578947,162.381579,2.342105,0.0
std,227.681667,3.936633,0.830698,6.968601,33.670575,30.833262,130.941976,90.236466,13.765794,147.556491,...,0.772328,0.567131,0.611775,1.169795,6.526599,216.461158,19.322025,42.195508,2.211414,0.0
min,545.000000,0.000000,0.000000,0.000000,1.000000,0.000000,59.000000,7.000000,1.000000,82.000000,...,1.000000,1.000000,1.000000,0.000000,18.000000,16.000000,0.500000,107.750000,0.000000,0.0
25%,626.500000,1.000000,0.000000,0.000000,5.000000,0.000000,175.000000,41.500000,4.500000,171.500000,...,2.000000,2.000000,2.000000,1.000000,18.000000,92.000000,2.750000,131.625000,0.500000,0.0
50%,754.000000,3.000000,0.000000,1.000000,9.000000,1.000000,204.000000,79.000000,9.000000,298.000000,...,3.000000,2.000000,3.000000,1.000000,19.000000,115.000000,5.000000,157.250000,2.000000,0.0
75%,904.000000,6.500000,1.000000,5.500000,18.000000,4.500000,274.500000,129.000000,18.500000,380.500000,...,3.000000,2.000000,3.000000,3.000000,21.000000,373.500000,14.750000,186.250000,3.750000,0.0
max,1444.000000,14.000000,2.000000,22.000000,146.000000,136.000000,604.000000,335.000000,45.000000,684.000000,...,3.000000,3.000000,3.000000,3.000000,42.000000,740.000000,84.000000,252.250000,8.000000,0.0


In [60]:
df_cluster_1 = df_agrupado_soma[df_agrupado_soma['cluster'] == 1]
df_cluster_1.describe()

,quantPosts,somaProfile,somaCover,somaAddPhotoWithOthers,somaIsWithOthers,somaAddPhoto,somaSharedPhoto,somaSharedVideo,somaSharedLink,somaSharedPost,...,apetite,concentracao,fadiga,int_sexo,idade,somaLikes,mediaInteracoesComOutros,mediaSharedPhotoPostVideo,mediaAtualizacaoPerfil,cluster
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,...,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.0
mean,244.000000,3.241379,1.551724,4.965517,15.724138,2.275862,68.724138,35.655172,4.000000,55.931034,...,2.448276,2.068966,2.724138,1.241379,20.172414,116.586207,10.344828,41.077586,2.396552,1.0
std,122.850083,2.707861,1.478083,5.335078,17.094060,2.250342,64.750563,36.095385,4.898979,40.482740,...,0.909718,0.842235,0.527565,1.214648,5.650973,122.307726,9.922110,25.536835,1.744273,0.0
min,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,5.000000,0.000000,7.000000,...,0.000000,0.000000,1.000000,0.000000,18.000000,5.000000,0.000000,13.000000,0.000000,1.0
25%,141.000000,1.000000,0.000000,2.000000,3.000000,0.000000,29.000000,15.000000,0.000000,24.000000,...,2.000000,2.000000,3.000000,0.000000,18.000000,53.000000,4.500000,18.750000,1.500000,1.0
50%,193.000000,3.000000,1.000000,3.000000,11.000000,2.000000,46.000000,23.000000,3.000000,43.000000,...,3.000000,2.000000,3.000000,1.000000,18.000000,65.000000,7.000000,34.000000,2.500000,1.0
75%,317.000000,4.000000,2.000000,7.000000,23.000000,3.000000,69.000000,38.000000,6.000000,88.000000,...,3.000000,3.000000,3.000000,2.000000,20.000000,113.000000,15.500000,52.500000,3.000000,1.0
max,510.000000,10.000000,5.000000,25.000000,69.000000,9.000000,258.000000,153.000000,22.000000,133.000000,...,3.000000,3.000000,3.000000,3.000000,48.000000,570.000000,40.000000,105.000000,7.500000,1.0


In [61]:
df_cluster_2 = df_agrupado_soma[df_agrupado_soma['cluster'] == 2]
df_cluster_2.describe()

,quantPosts,somaProfile,somaCover,somaAddPhotoWithOthers,somaIsWithOthers,somaAddPhoto,somaSharedPhoto,somaSharedVideo,somaSharedLink,somaSharedPost,...,apetite,concentracao,fadiga,int_sexo,idade,somaLikes,mediaInteracoesComOutros,mediaSharedPhotoPostVideo,mediaAtualizacaoPerfil,cluster
count,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.00000,4.000000,4.000000,4.00000,4.000000,4.000000,4.00000,4.000000,4.000000,4.0
mean,436.500000,21.000000,7.000000,6.25000,16.750000,8.000000,131.500000,43.750000,12.500000,105.750000,...,2.50000,1.500000,1.750000,0.50000,20.250000,113.250000,11.50000,73.375000,14.000000,2.0
std,106.158686,8.406347,0.816497,9.17878,12.230427,8.164966,93.770998,33.737961,7.234178,50.367814,...,0.57735,1.290994,0.957427,0.57735,2.629956,67.859536,10.68488,34.546165,4.102845,0.0
min,318.000000,12.000000,6.000000,1.00000,9.000000,0.000000,51.000000,17.000000,4.000000,51.000000,...,2.00000,0.000000,1.000000,0.00000,18.000000,24.000000,5.50000,34.500000,10.000000,2.0
25%,375.000000,16.500000,6.750000,1.75000,10.500000,1.500000,58.500000,18.500000,7.750000,69.000000,...,2.00000,0.750000,1.000000,0.00000,18.750000,81.000000,5.87500,49.687500,11.500000,2.0
50%,430.500000,20.000000,7.000000,2.00000,11.500000,8.000000,113.000000,34.500000,13.500000,110.500000,...,2.50000,1.500000,1.500000,0.50000,19.500000,125.000000,6.50000,75.500000,13.250000,2.0
75%,492.000000,24.500000,7.250000,6.50000,17.750000,14.500000,186.000000,59.750000,18.250000,147.250000,...,3.00000,2.250000,2.250000,1.00000,21.000000,157.250000,12.12500,99.187500,15.750000,2.0
max,567.000000,32.000000,8.000000,20.00000,35.000000,16.000000,249.000000,89.000000,19.000000,151.000000,...,3.00000,3.000000,3.000000,1.00000,24.000000,179.000000,27.50000,108.000000,19.500000,2.0
